In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

## Dataset

In [3]:
df=pd.read_csv("/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv", index_col=0)
df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
id,,,,,,,,,,,
1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [4]:
df['stops'] = df['stops'].replace({'one': 1, 'zero': 0, 'two_or_more': 2})
df['class'] = df['class'].replace({'Economy': 0, 'Business': 1})
df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
id,,,,,,,,,,,
1,Vistara,UK-810,Bangalore,Early_Morning,1,Night,Mumbai,0,14.25,21,7212
2,SpiceJet,SG-5094,Hyderabad,Evening,0,Night,Kolkata,0,1.75,7,5292
3,Vistara,UK-846,Bangalore,Morning,1,Evening,Delhi,1,9.58,5,60553
4,Vistara,UK-706,Kolkata,Morning,1,Evening,Hyderabad,0,6.75,28,5760
5,Indigo,6E-5394,Chennai,Early_Morning,0,Morning,Mumbai,0,2.00,4,10712


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 1 to 20000
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   airline           20000 non-null  object 
 1   flight            20000 non-null  object 
 2   source_city       20000 non-null  object 
 3   departure_time    20000 non-null  object 
 4   stops             20000 non-null  int64  
 5   arrival_time      20000 non-null  object 
 6   destination_city  20000 non-null  object 
 7   class             20000 non-null  int64  
 8   duration          20000 non-null  float64
 9   days_left         20000 non-null  int64  
 10  price             20000 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 1.8+ MB


## Preparation for ML

In [6]:
X_train=df.drop('price', axis=1)
y=df['price']

train_set, test_set=train_test_split(df, test_size=0.2, random_state=42)

In [7]:
num_attribs=['stops', 'class', 'duration', 'days_left']
cat_attribs=['airline', 'flight', 'source_city', 'departure_time', 'arrival_time', 'destination_city']

num_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', cat_pipeline, cat_attribs)
])

In [8]:
X_prepared = full_pipeline.fit_transform(X_train)
X_prepared.toarray()[0:5,:]

array([[ 0.19281382, -0.67396452,  0.28952803, ...,  0.        ,
         0.        ,  1.        ],
       [-2.30154354, -0.67396452, -1.45682723, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.19281382,  1.48375764, -0.36291029, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.19281382, -0.67396452, -0.75828512, ...,  1.        ,
         0.        ,  0.        ],
       [-2.30154354, -0.67396452, -1.42190012, ...,  0.        ,
         0.        ,  1.        ]])

## ML
### Linear Regression

In [9]:
LR_model=LinearRegression()
LR_model.fit(X_prepared, y)

LinearRegression()

In [10]:
predict = LR_model.predict(X_prepared)

lin_mse=mean_squared_error(y, predict)
lin_rmse=np.sqrt(lin_mse)
print(lin_rmse)

6112.051554417245


In [11]:
X_test=test_set.drop('price', axis=1)
y_test=test_set['price'].copy()

In [12]:
X_test_prepared = full_pipeline.transform(X_test)

predict = LR_model.predict(X_test_prepared)

lin_mse=mean_squared_error(y_test, predict)
lin_rmse=np.sqrt(lin_mse)
print(lin_rmse)

6063.987317490167


### Decision Tree Regression

In [13]:
Tree_model = DecisionTreeRegressor()
Tree_model.fit(X_prepared, y)

DecisionTreeRegressor()

In [14]:
y_predicted = Tree_model.predict(X_test_prepared)

lin_mse=mean_squared_error(y_test, y_predicted)
lin_rmse=np.sqrt(lin_mse)
print(lin_rmse)

13.858681845687922


In [15]:
pd.DataFrame({'Predict':y_predicted, 'Real price':y_test})

,Predict,Real price
id,,
10651,5361.0,5361
2042,50297.0,50297
8669,60232.0,60232
1115,74365.0,74365
13903,4417.0,4417
...,...,...
4074,10228.0,10228
7443,12544.0,12544
10000,5276.0,5276
